In [1]:
def accuracy(y_pred, y_true):
    true_lab = np.sum(((y_pred - y_true) == 0).astype(np.int))
    accu = true_lab / y_pred.size
    return accu

def get_confusion_matrix(y_pred, y_true):
    matrix = np.zeros((3,3))
    for i in range(0, y_pred.size):
        matrix[y_true[i], y_pred[i]] += 1
    return matrix

def softmax(score: np.ndarray) -> np.ndarray:
    max_values = score.max(axis=1)
    e = np.exp(score - max_values[:,np.newaxis])
    return e / e.sum(axis=1)[:,np.newaxis]

def one_hot_labels(Y: np.array) -> np.ndarray:
    # calculate number of classes
    number_of_classes = len(np.unique(Y))
    # build one hot encoded labels
    return np.eye(number_of_classes)[Y]

def cross_entropy_loss(Y_one_hot: np.ndarray, Y_proba: np.ndarray) -> float:
    m = Y_one_hot.shape[0]
    
    log_likelihood = -np.log(Y_proba[range(m), Y_one_hot.argmax(axis=1)])
    loss = np.sum(log_likelihood) / m
    return loss

def get_loss_with_grad(W, X, Y, lambd, regularization):
    m = X.shape[0]
    scores = np.dot(X, W)
    prob = softmax(scores)
    
    reg_loss, reg_grad = regularization(W, lambd)
    
    loss = (-1 / m) * np.sum(Y * np.log(prob)) + reg_loss
    grad = (-1 / m) * np.dot(X.T, (Y - prob)) + reg_grad
    return loss, grad

def L1(W, lambd) -> (float, float):
    """
    return L1 and according derivative
    """
    return lambd * np.sum(np.abs(W)), W / np.abs(W) 

def L2(W, lambd) -> (float, float):
    """
    return L2 and according derivative
    """
    return (lambd / 2) * np.sum(W ** 2), lambd * W

def no_reg(W, lambd):
    return 0

def shuffle(X, Y):
    """
    Shuffle X and Y
    """
    p = np.random.permutation(len(X))
    return X[p, ...], Y[p, ...]

def split_dataset(X, Y, val_ratio=0.1):
    """
    Split dataset into train and val sets in given ratio
    """
    train_end = int(len(X) * (1 - val_ratio))
    return X[:train_end, ...], Y[:train_end, ...], X[train_end:, ...], Y[train_end:, ...]

def add_bias(X):
    """
    Add bias to features
    """
    return np.c_[X, np.ones(len(X))]    

def select_regularization(regularizer: str):
        if regularizer == 'l1':
            regularization = L1
        elif regularizer == 'l2':
            regularization = L2
        elif regularizer is None:
            regularization = no_reg
        else:
            raise ValueError(f'Unknown regularizer value: {regularizer}')
        return regularization

NameError: name 'np' is not defined

In [ ]:
class Softmax_Regression:
    def __init__(self):
        np.random.seed(0)
    
        
    def fit(self, X, Y, learning_rate=0.01, epochs=1000, tol=None, regularizer=None,
            lambd=0.0, early_stopping=False, validation_fraction=0.1, **kwargs):
        
        self.epochs = epochs
        self.tol = tol
        self.early_stopping = early_stopping
        self.learning_rate = learning_rate
        self.lambd = lambd
        # add bias term
        X = add_bias(X)
        Y = one_hot_labels(Y)
        
        # initial weights
        self._initialize_weigts()
        
        # select regularization method
        regularization = select_regularization(regularizer)
        
        if early_stopping:
            self.best_loss = np.inf
            self.val_losses = []
            
            # split into train and val
            X_train, Y_train, X_val, Y_val = split_dataset(*shuffle(X, Y))
            return self.learn_with_early_stop(X_train, Y_train, X_val, Y_val, regularization)
        else:
            self.X_train, self.Y_train = X, Y
            return self.learn(X, Y, regularization)
        
    def predict(self, X):
        return np.argmax(softmax(add_bias(X) @ self.theta), axis=1)
        
    def learn(self, X_train, Y_train, regularization):
        self.losses = []
        for x in range(self.epochs):
            loss = self._grad_step(X_train, Y_train, regularization)
            self.losses.append(loss)
            
            if self.is_tol():
#                 print('Stop due to tol')
                break
        return self.losses
    
    def learn_with_early_stop(self, X_train, Y_train, X_val, Y_val, regularization):
        """
        Gradient descent with early stoping
        """
        self.losses = []
        for x in range(self.epochs):
            loss = self._grad_step(X_train, Y_train, regularization)
            self.losses.append(loss)
            
            val_loss, _ = get_loss_with_grad(self.theta, X_val, Y_val, self.lambd, regularization)
            self.val_losses.append(val_loss)
            if val_loss < self.best_loss:
                self.best_loss = val_loss
            else:
#                 print('Early stop. Validation score is not improving')
                break
                
            if self.is_tol():
#                 print('Stop due to tol')
                break
                
        return self.losses
        
    
    def is_tol(self):
        if self.tol and len(self.losses) > 1 and self.tol > np.abs(self.losses[-1] - self.losses[-2]):
            return True
        return False
        
    def _initialize_weigts(self):
        self.theta = np.random.random((X.shape[1] + 1, len(np.unique(Y))))
    
    def _grad_step(self, X, Y, regularization):
        loss, grad = get_loss_with_grad(self.theta, X, Y, self.lambd, regularization)
        self.theta -= self.learning_rate * grad
        return loss
        


In [ ]:
class Softmax_SGD(Softmax_Regression):
    def __init__(self):
        super().__init__()
        
    def learn(self, X_train, Y_train, regularization):
        self.losses = []
        for x in range(self.epochs):
            batch_losses = []
            for x, y in zip(X_train, Y_train):
                loss = self._grad_step(x[np.newaxis, ...], y, regularization)
                batch_losses.append(loss)
            losses.append(np.mean(batch_losses))

            if self.is_tol():
#                 print('Stop due to tol')
                break
        return losses
    
    def learn_with_early_stop(self, X_train, Y_train, X_val, Y_val, regularization):
        """
        Gradient descent with early stoping
        """
        self.losses = []
        for x in range(self.epochs):
            batch_losses = []
            for x, y in zip(X_train, Y_train):
                loss = self._grad_step(X_train, Y_train, regularization)
                batch_losses.append(loss)
            self.losses.append(np.mean(batch_losses))
            
            val_loss, _ = get_loss_with_grad(self.theta, X_val, Y_val, self.lambd, regularization)
            self.val_losses.append(val_loss)
            if val_loss < self.best_loss:
                self.best_loss = val_loss
            else:
#                 print('Early stop. Validation score is not improving')
                break

            if self.is_tol():
#                 print('Stop due to tol')
                break
                
        return self.losses

In [ ]:
def kFold(folds, data, labels, model, model_args, error_function):
    error_train = []
    error_validation = []
    nrows = data.shape[0]
    # we divide the dataset in equal parts. If there are extra rows, we ignore them
    f_size = int(np.floor(nrows/folds))
    data, labels = data[0:(f_size * folds)], labels[0:(f_size * folds)]
    data, labels = np.split(data, folds), np.split(labels, folds)

    expected_labels = np.ndarray([])
    predicted_labels = np.ndarray([])
    
    for fold in range(folds):
        test_created = False
        for i in range(folds):
            if fold == i:
                X_test = data[i]
                y_test = labels[i]
                test_created = True
            else:
                if i==0 or (i == 1 and test_created):
                    X_train = data[i]
                    y_train = labels[i]
                else:
                    X_train = np.append(X_train, data[i], axis=0)
                    y_train = np.append(y_train, labels[i], axis=0)
        # test and train sets have been created
        
        reg_model = model()
        reg_model.fit(X_train, y_train, learning_rate=model_args['learning_rate'], tol=model_args['tol'], lambd=model_args['lambd'], regularizer=model_args['regularizer'])
        
        y_pred_train = reg_model.predict(X_train)
        y_pred_validation = reg_model.predict(X_test)
        
        expected_labels = np.append(expected_labels, y_test)
        predicted_labels = np.append(predicted_labels, y_pred_validation)
        error_train.append(error_function(y_pred_train, y_train))
        error_validation.append(error_function(y_pred_validation, y_test))
        error = {'error_train': np.mean(error_train), 'error_validation': np.mean(error_validation)}
    return {'expected_labels': expected_labels, 'predicted_labels': predicted_labels, 'average_error': error}
